In [1]:
import re
from unsloth import FastLanguageModel
from tabulate import tabulate

model_id = "MahdiMaaref/Persian-To-English-Translator"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_id,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

persian_texts = [
    "امروز بعد از کار می‌خوام برم کتابخانه و چند تا کتاب خوب پیدا کنم.",
    "هوا بارونه، حتماً چترت رو با خودت بیار.",
    "جلسه‌ی فردا صبح ساعت ۱۰ برگزار می‌شه، برنامه تغییر کرده.",
    "اگه فرصت کردی، لطفاً پیش‌نویس گزارش رو قبل از ظهر برام بفرست.",
    "من عاشق سفر به شهرهای تاریخی‌ام و همیشه عکس می‌گیرم.",
    "برای خرید مواد غذایی، بهتره لیستت رو آماده کنی که چیزی یادت نره.",
    "آخر هفته دوست دارم برم یه کافه‌ی دنج، کتاب بخونم و چای بخورم."
]


results = []

for persian_text in persian_texts:
    messages = [
        {"role": "system", "content": "You are a professional Persian to English translator. Translate accurately. Output ONLY English."},
        {"role": "user", "content": f"Translate to English:\n{persian_text}"}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=2048,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
    )

    
    input_len = inputs['input_ids'].shape[1]
    new_tokens = outputs[0][input_len:]
    translation = tokenizer.decode(new_tokens, skip_special_tokens=True)
    translation = re.sub(r'<think>.*?</think>', '', translation, flags=re.DOTALL).strip()
    results.append([persian_text, translation])

print(tabulate(results, headers=["Persian", "English Translation"], tablefmt="github"))

ModuleNotFoundError: No module named 'unsloth'